Data preparation

In [25]:
import json
import pandas as pd
import numpy as np
import os

np.random.seed(0)

In [26]:
DATA_DIR = "../data"
DATA_JSON = os.path.join(DATA_DIR, "data.json") # Path to data.json
DATA_INFO = os.path.join(DATA_DIR, "data.info") # Path to data.info

In [27]:
labels_df = pd.read_csv(DATA_INFO)
labels_dict = {}

for _, row in labels_df.iterrows():
  labels_dict[row['transcript_id']] = labels_dict.get(row['transcript_id'], {})
  labels_dict[row['transcript_id']][row['transcript_position']] = row['label']

In [28]:
unique_gene_id = list(labels_df.gene_id.unique())

In [29]:
# Split gene ID for training and testing

n = len(unique_gene_id)
train_gene_id = unique_gene_id[:int(0.8 * len(unique_gene_id))]
test_gene_id = unique_gene_id[int(0.8 * len(unique_gene_id)):]

In [30]:
train_labels_df = labels_df[labels_df['gene_id'].isin(train_gene_id)]
test_labels_df = labels_df[labels_df['gene_id'].isin(test_gene_id)]

train0, train1 = train_labels_df["label"].value_counts()
test0, test1 = test_labels_df["label"].value_counts()

In [31]:
print(f'Train class ratio {int(train0/train1)}:1') # 21:1
print(f'Test class ratio {int(test0/test1)}:1') # 20:1

Train class ratio 20:1
Test class ratio 27:1


In [32]:
instance_lists = []

with open(DATA_JSON) as f:
  for transcript_json in f:
    transcript_dict = json.loads(transcript_json)
    for transcript_id, transcript_pos_dict in transcript_dict.items():
      for transcript_pos, nucleotides_dict in transcript_pos_dict.items():
        for nucleotides, data in nucleotides_dict.items():
          for row in data:
            instance_lists.append([transcript_id, transcript_pos, nucleotides] + row + [labels_dict[transcript_id][int(transcript_pos)]])

In [33]:
complete_df = pd.DataFrame(instance_lists, columns=['transcript_id', 'transcript_position', 'nucleotides', '0', '1', '2', '3', '4', '5', '6', '7', '8','label'])

complete_df.head()

,transcript_id,transcript_position,nucleotides,0,1,2,3,4,5,6,7,8,label
0,ENST00000000233,244,AAGACCA,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,0
1,ENST00000000233,244,AAGACCA,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,0
2,ENST00000000233,244,AAGACCA,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,0
3,ENST00000000233,244,AAGACCA,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,0
4,ENST00000000233,244,AAGACCA,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,0


In [34]:
complete_df_mean = complete_df.groupby(by=['transcript_id', 'transcript_position', 'nucleotides']).mean().reset_index()

complete_df_mean.head()

,transcript_id,transcript_position,nucleotides,0,1,2,3,4,5,6,7,8,label
0,ENST00000000233,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,4.386989,80.570270,0.0
1,ENST00000000233,261,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,3.016599,94.290698,0.0
2,ENST00000000233,316,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,2.087146,89.364324,0.0
3,ENST00000000233,332,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,2.236520,89.154000,0.0
4,ENST00000000233,368,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,4.260253,85.178788,0.0


In [35]:
complete_df_min = complete_df.groupby(by=['transcript_id', 'transcript_position', 'nucleotides']).min().reset_index()
complete_df_min.columns = ['transcript_id', 'transcript_position', 'nucleotides', '9', '10', '11', '12', '13', '14', '15', '16', '17', 'label']

complete_df_min.head()

,transcript_id,transcript_position,nucleotides,9,10,11,12,13,14,15,16,17,label
0,ENST00000000233,244,AAGACCA,0.00199,1.770,102.0,0.00232,1.040,111.0,0.00232,0.773,73.1,0
1,ENST00000000233,261,CAAACTG,0.00199,0.919,98.3,0.00166,0.789,96.1,0.00232,0.715,88.6,0
2,ENST00000000233,316,GAAACAG,0.00232,1.280,101.0,0.00166,1.020,88.7,0.00199,0.630,84.4,0
3,ENST00000000233,332,AGAACAT,0.00232,1.220,110.0,0.00283,1.400,93.5,0.00199,0.884,81.4,0
4,ENST00000000233,368,AGGACAA,0.00199,1.150,110.0,0.00232,2.680,112.0,0.00266,1.040,77.6,0


In [36]:
complete_df_max = complete_df.groupby(by=['transcript_id', 'transcript_position', 'nucleotides']).max().reset_index()
complete_df_max.columns = ['transcript_id', 'transcript_position', 'nucleotides', '18', '19', '20', '21', '22', '23', '24', '25', '26', 'label']

complete_df_max.head()

,transcript_id,transcript_position,nucleotides,18,19,20,21,22,23,24,25,26,label
0,ENST00000000233,244,AAGACCA,0.0339,13.4,132.0,0.0296,15.70,133.0,0.0329,15.50,88.3,0
1,ENST00000000233,261,CAAACTG,0.0222,17.0,115.0,0.0267,11.50,116.0,0.0262,14.10,103.0,0
2,ENST00000000233,316,GAAACAG,0.0299,11.6,110.0,0.0349,6.23,107.0,0.0266,6.85,96.2,0
3,ENST00000000233,332,AGAACAT,0.0370,14.2,136.0,0.0286,16.90,106.0,0.0214,6.49,95.7,0
4,ENST00000000233,368,AGGACAA,0.0478,39.0,125.0,0.0329,13.50,131.0,0.0485,8.81,90.5,0


In [37]:
complete_df_all = complete_df_mean.drop(columns=['label']).merge(complete_df_min.drop(columns=['label'])).merge(complete_df_max.drop(columns=['label']))
complete_df_all['transcript_position'] = complete_df_all['transcript_position'].astype('int')

complete_df_all.head()

,transcript_id,transcript_position,nucleotides,0,1,2,3,4,5,6,...,17,18,19,20,21,22,23,24,25,26
0,ENST00000000233,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,...,73.1,0.0339,13.4,132.0,0.0296,15.70,133.0,0.0329,15.50,88.3
1,ENST00000000233,261,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,...,88.6,0.0222,17.0,115.0,0.0267,11.50,116.0,0.0262,14.10,103.0
2,ENST00000000233,316,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,...,84.4,0.0299,11.6,110.0,0.0349,6.23,107.0,0.0266,6.85,96.2
3,ENST00000000233,332,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,...,81.4,0.0370,14.2,136.0,0.0286,16.90,106.0,0.0214,6.49,95.7
4,ENST00000000233,368,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,...,77.6,0.0478,39.0,125.0,0.0329,13.50,131.0,0.0485,8.81,90.5


In [38]:
complete_df_all = complete_df_all.merge(labels_df, on=['transcript_id', 'transcript_position'])

complete_df_all.head()

,transcript_id,transcript_position,nucleotides,0,1,2,3,4,5,6,...,19,20,21,22,23,24,25,26,gene_id,label
0,ENST00000000233,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,...,13.4,132.0,0.0296,15.70,133.0,0.0329,15.50,88.3,ENSG00000004059,0
1,ENST00000000233,261,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,...,17.0,115.0,0.0267,11.50,116.0,0.0262,14.10,103.0,ENSG00000004059,0
2,ENST00000000233,316,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,...,11.6,110.0,0.0349,6.23,107.0,0.0266,6.85,96.2,ENSG00000004059,0
3,ENST00000000233,332,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,...,14.2,136.0,0.0286,16.90,106.0,0.0214,6.49,95.7,ENSG00000004059,0
4,ENST00000000233,368,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,...,39.0,125.0,0.0329,13.50,131.0,0.0485,8.81,90.5,ENSG00000004059,0


In [55]:
train_df = complete_df_all[complete_df_all['gene_id'].isin(train_gene_id)]
test_df = complete_df_all[complete_df_all['gene_id'].isin(test_gene_id)]

In [56]:
# label_counts = labels_df.label.value_counts()
# label_ratio = label_counts[0]/label_counts[1]

label_ratio = train0/train1

label_ratio

20.338048679009777

Model training and evaluation

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_curve, precision_recall_curve, auc
from scipy.stats import mode

In [58]:
def get_roc_auc(y_true, y_pred):
    fpr, tpr, _  = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)
    return roc_auc


def get_pr_auc(y_true, y_pred):
    precision, recall, _ = precision_recall_curve(y_true, y_pred, pos_label=1)
    pr_auc = auc(recall, precision)
    return pr_auc


def get_accuracy(y_true, y_pred):
    return balanced_accuracy_score(y_true, y_pred)

In [44]:
!pip install xgboost==1.6.2

In [45]:
from xgboost import XGBClassifier
from math import ceil

In [ ]:
# To make sure our model is robust against unseen data, we perform multiple iterations of model training and evaluation
# Each time, we split the data into training and evaluation set by gene ID

roc_auc = []
pr_auc = []
accuracy = []

for i in range(len(unique_gene_id)):
  print(f'Inspecting gene {i+1}/{len(unique_gene_id)}')
  gene_id = unique_gene_id[i]
  train_df = complete_df_all[complete_df_all['gene_id'] != gene_id].iloc[:, 3:-2]
  train_label = complete_df_all[complete_df_all['gene_id'] != gene_id].iloc[:, -1]
  eval_df = complete_df_all[complete_df_all['gene_id'] != gene_id].iloc[:, 3:-2]
  eval_label = complete_df_all[complete_df_all['gene_id'] != gene_id].iloc[:, -1]
  
  xgb_model = XGBClassifier(
      objective = 'binary:logistic',
      scale_pos_weight = ceil(label_ratio),
      max_delta_step = 1,
      seed = 0
      # tree_method = 'gpu_hist'
  )

  xgb_model.fit(train_df, train_label)
  predictions = xgb_model.predict(eval_df)

  roc_auc.append(get_roc_auc(eval_label.to_numpy(), predictions))
  pr_auc.append(get_pr_auc(eval_label.to_numpy(), predictions))
  accuracy.append(get_accuracy(eval_label.to_numpy(), predictions))

In [ ]:
print(f'ROC AUC: {np.mean(roc_auc)}') # ROC AUC: 0.9392459265323052
print(f'PR AUC: {np.mean(pr_auc)}') # PR AUC: 0.6521694199992294
print(f'Accuracy: {np.mean(accuracy)}') # Accuracy: 0.9392459265323052

In [ ]:
# Do the same thing now, but we use mean only to compare the performances of the two models

complete_df_mean['transcript_position'] = complete_df_mean['transcript_position'].astype('int')
complete_df_mean = complete_df_mean.drop(columns=['label']).merge(labels_df, on=['transcript_id', 'transcript_position'])

complete_df_mean.head()

In [ ]:
# Mean only

roc_auc = []
pr_auc = []
accuracy = []

for i in range(len(unique_gene_id)):
  print(f'Inspecting gene {i+1}/{len(unique_gene_id)}')
  gene_id = unique_gene_id[i]
  train_df = complete_df_mean[complete_df_mean['gene_id'] != gene_id].iloc[:, 3:-2]
  train_label = complete_df_mean[complete_df_mean['gene_id'] != gene_id].iloc[:, -1]
  eval_df = complete_df_mean[complete_df_mean['gene_id'] != gene_id].iloc[:, 3:-2]
  eval_label = complete_df_mean[complete_df_mean['gene_id'] != gene_id].iloc[:, -1]
  
  xgb_model = XGBClassifier(
      objective = 'binary:logistic',
      scale_pos_weight = ceil(label_ratio),
      max_delta_step = 1,
      seed = 0
      # tree_method = 'gpu_hist'
  )

  xgb_model.fit(train_df, train_label)
  predictions = xgb_model.predict(eval_df)

  roc_auc.append(get_roc_auc(eval_label.to_numpy(), predictions))
  pr_auc.append(get_pr_auc(eval_label.to_numpy(), predictions))
  accuracy.append(get_accuracy(eval_label.to_numpy(), predictions))

In [ ]:
print(f'ROC AUC: {np.mean(roc_auc)}') # ROC AUC: 0.913483812474404
print(f'ROC AUC: {np.mean(pr_auc)}') # PR AUC: 0.6115034727496196
print(f'ROC AUC: {np.mean(accuracy)}') # Accuracy: 0.913483812474404

In [ ]:
# Train model with the full dataset and save it

xgb_model = XGBClassifier(
    objective = 'binary:logistic',
    scale_pos_weight = ceil(label_ratio),
    max_delta_step = 1,
    seed = 0,
    tree_method = 'gpu_hist'
)

xgb_model.fit(complete_df_all.iloc[:, 3:-2], complete_df_all.iloc[:, -1])
predictions = xgb_model.predict(complete_df_all.iloc[:, 3:-2])

print(f'ROC AUC: {get_roc_auc(complete_df_all.iloc[:, -1].to_numpy(), predictions)}') # 0.9389924569614602
print(f'PR AUC: {get_pr_auc(complete_df_all.iloc[:, -1].to_numpy(), predictions)}') # 0.6503314477067172
print(f'Accuracy: {get_accuracy(complete_df_all.iloc[:, -1].to_numpy(), predictions)}') # 0.9389924569614602

In [60]:
# Train model with the train dataset and evaluate it using test dataset

xgb_model = XGBClassifier(
    objective = 'binary:logistic',
    scale_pos_weight = ceil(label_ratio),
    max_delta_step = 1,
    seed = 0
    # tree_method = 'gpu_hist'
)

xgb_model.fit(train_df.iloc[:, 3:-2], train_df.iloc[:, -1])
predictions = xgb_model.predict(test_df.iloc[:, 3:-2])

print(f'ROC AUC: {get_roc_auc(test_df.iloc[:, -1].to_numpy(), predictions)}') # 0.7973242302264072
print(f'PR AUC: {get_pr_auc(test_df.iloc[:, -1].to_numpy(), predictions)}') # 0.4808366720979996
print(f'Accuracy: {get_accuracy(test_df.iloc[:, -1].to_numpy(), predictions)}') # 0.7973242302264072

ROC AUC: 0.806509923480279
PR AUC: 0.4646358890000649
Accuracy: 0.806509923480279


## Save model

In [61]:
import os
MODEL_DIR = os.path.join(os.path.expanduser('~'), 'studies/ProjectStorage/tanyeejet/models')
xgb_model.save_model(os.path.join(MODEL_DIR, 'xgb4_3.model'))

In [62]:
test_df.iloc[:, 3:-2].to_csv('../data/xgb3_test_feature.csv', index=False)
test_df.iloc[:, -1].to_csv('../data/xgb3_test_label.csv', index=False)

In [64]:
test_df.iloc[:, 3:-2].head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
85009,0.009085,6.718889,124.370370,0.006952,3.166296,101.922222,0.008211,2.552963,95.062963,0.00232,...,89.5,0.0253,10.00,134.0,0.0130,8.68,113.0,0.0220,4.86,102.0
85010,0.008538,3.131034,107.655172,0.008944,3.298966,106.241379,0.006550,2.458621,95.906897,0.00232,...,91.0,0.0189,8.65,111.0,0.0266,5.09,110.0,0.0176,9.30,103.0
85011,0.009150,2.986667,106.466667,0.007278,3.861000,100.106667,0.004990,1.645200,86.303333,0.00258,...,80.6,0.0189,8.86,111.0,0.0183,5.72,109.0,0.0108,3.33,89.1
85012,0.008693,6.311613,124.967742,0.008585,2.846774,99.522581,0.005066,2.260645,94.087097,0.00232,...,79.7,0.0252,11.70,131.0,0.0302,10.00,103.0,0.0143,4.97,100.0
85013,0.005759,3.031071,118.857143,0.010269,5.787500,121.714286,0.006084,3.398571,81.664286,0.00199,...,74.8,0.0169,7.34,123.0,0.0322,12.50,126.0,0.0176,8.69,88.1


In [65]:
test_df.iloc[:, -1].head()

85009    0
85010    0
85011    0
85012    0
85013    0
Name: label, dtype: int64